In [1]:
import fs from 'fs'
import xlsx from 'node-xlsx';
import { format } from 'date-fns';

In [3]:
enum Classification {
  BCGEU = 'BCGEU',
  EXCLUDED = 'EXCLUDED',
  BCNU = 'BCNU',
  PEA = 'PEA',
}

In [58]:
interface Personnel {
    firstName: string;
    lastName: string;
    region: string;
    workLocation: string;
    ministry: string;
    primaryPhone: string;
    secondaryPhone: string;
    otherPhone: string;
    email: string;
    applicationDate: string;
    skillsAbilities: string;
    notes: string;
    supervisor: string;
    active: boolean;
    classification: Classification;
    remoteOnly: boolean;
    willingToTravel: boolean;
    experiences: { function: number, level: string }[];
}

In [5]:
const numHeaderRows = 4;
const numEndRows = 19;

const worksheet = xlsx.parse(fs.readFileSync(`teamsmembers.xlsx`));
const worksheetArrays = worksheet.find(page => page.name === 'Active TEAMS Members')?.data || [];
const members = worksheetArrays.slice(numHeaderRows, worksheetArrays.length-numEndRows);

In [6]:
const getApplicationDate = (date: number | string | undefined): string => {
 if (date === '<2018') {
    return format(new Date('2018-01-01'), 'P');
 } else if (!date) {
    return format(new Date('2024-01-01'), 'P');
 } else if (typeof date === 'number') {
    return format(new Date(1900, 0, date-1), 'P');
 } else {
    return format(new Date('2024-01-01'), 'P');
 }
}

In [8]:
const getClassification = (notesValue: string): Classification => {
    if (notesValue?.includes('BCGEU')) {
        return Classification.BCGEU;
    } else {
        return Classification.EXCLUDED;
    }
}

In [65]:
// We only need this if the excel spreadsheet order of functions IS NOT the same as on the database
// OR if the db ids of the functions are not auto increment integers starting from 1
const getFunction = (i: number): string => {
    switch (i) {
        case 0: return 'Ops';
        case 1: return 'ESS';
        case 2: return 'FN';
        case 3: return 'Fin';
        case 4: return 'Liaison';
        case 5: return 'Logs';
        case 6: return 'Plans';
        case 7: return 'APU';
        case 8: return 'Recovery';
        case 9: return 'DDir';
        case 10: return 'GIS';
        default: return '';
    }
}

const getExperienceLevel = (e: string): string => {
    switch (e) {
        case 'I': return 'INTERESTED';
        case 'X': return 'EXPERIENCED';
        case 'C': return 'CHIEF_EXPERIENCED';
        case 'O': return 'OUTSIDE_EXPERIENCED';
        default: return '';
    }
}

In [83]:
const getExperiences = (rawExperiences: (string | undefined)[]) => {
    const experiences = [];
    for (let i = 0; i < rawExperiences.length; i++) {
        if (!!rawExperiences[i] && ['C','X','I','O'].includes(rawExperiences[i])) {
            // experiences.push({ function: getFunction(i), level: rawExperiences[i] });
            experiences.push({ function: i+1, level: getExperienceLevel(rawExperiences[i]) });
        }
    }
    return experiences;
}

In [84]:

const personnel: Personnel[] = members.map(member => {
    return {
        firstName: member[0].split(', ')[1],
        lastName: member[0].split(', ')[0],
        region: member[1],
        workLocation: member[2],
        ministry: member[3],
        primaryPhone: member[4],
        secondaryPhone: member[5],
        otherPhone: member[6],
        email: member[7],
        applicationDate: getApplicationDate(member[8]),
        skillsAbilities: member[32],
        notes: member[34],
        supervisor: member[33],
        classification: getClassification(member[34]), // Need function
        active: true,
        remoteOnly: false,
        willingToTravel: false,
        experiences: getExperiences(member.slice(11, 20)),
    };
});

In [93]:
const sqlPath = 'import-personnel.sql';
await fs.truncate(sqlPath, () => { console.log('deleted file') });

deleted file


In [94]:
for (const p of personnel) {
    await fs.appendFileSync(sqlPath, `\nINSERT INTO personnel ("first_name", "last_name", "work_location", "region", "ministry", "primary_phone", "secondary_phone", "other_phone", "email", "application_date", "supervisor", "skills_abilities", "notes", "active", "classification", "remote_only", "willing_to_travel")
    VALUES
    ('${p.firstName}', '${p.lastName}', '${p.workLocation}', '${p.region}', '${p.ministry}', ${p.primaryPhone || null}, ${p.secondaryPhone || null}, ${p.otherPhone || null}, '${p.email}', '${p.applicationDate}', '${p.supervisor}', '${p.skillsAbilities}', '${p.notes}', true, '${p.classification}', ${p.remoteOnly}, ${p.willingToTravel});
    `);
}

In [95]:
for (const p of personnel) {
    if (p.experiences.length > 0) {
        for (const [i, e] of p.experiences.entries()) {
            await fs.appendFileSync(sqlPath, `\nINSERT INTO personnel_function_experience ("personnel_id", "function_id", "experience_type") SELECT id, ${e.function} as function_id, '${e.level}' as experience_type FROM personnel WHERE email = '${p.email}';`);
        }
    }
}